# 05 – Transformación: playlist_items_uploads

Esta tabla contine todas los videos del canal con una playlist igual para todas (uploads) la cual es creada por el mismo canal.
No contiene métricas ni snapshots, solo la estructura editorial
del canal.


In [1]:
from pathlib import Path
import json
import pandas as pd
from datetime import datetime

PROJECT_ROOT = Path.cwd().parent
RAW_PATH = PROJECT_ROOT / "data" / "raw" / "youtube"
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

PROCESSED_PATH.mkdir(parents=True, exist_ok=True)


In [2]:
with open(RAW_PATH / "playlist_items_uploads.json", "r", encoding="utf-8") as f:
    playlist_items_raw = json.load(f)

len(playlist_items_raw["items"])


199

In [6]:
from datetime import datetime, timezone

# Timestamp explícitamente en UTC
EXTRACTED_AT = datetime.now(timezone.utc)

EXTRACTED_AT

datetime.datetime(2026, 2, 14, 22, 33, 26, 405853, tzinfo=datetime.timezone.utc)

In [10]:
rows = []

for item in playlist_items_raw["items"]:
    rows.append({
        "playlist_id": item["snippet"]["playlistId"],
        "video_id": item["contentDetails"]["videoId"],
        "position": item["snippet"]["position"],
        "added_at": item["snippet"]["publishedAt"],
        "extracted_at": EXTRACTED_AT
    })

df_playlist_items_uploads = pd.DataFrame(rows)
df_playlist_items_uploads.head()


,playlist_id,video_id,position,added_at,extracted_at
0,UUUEOHBht8pnQhQvCfIcl-gg,xB4ecIksJSY,0,2026-01-24T12:04:21Z,2026-02-14 22:33:26.405853+00:00
1,UUUEOHBht8pnQhQvCfIcl-gg,7bwkNrRpgw0,1,2026-01-23T06:52:23Z,2026-02-14 22:33:26.405853+00:00
2,UUUEOHBht8pnQhQvCfIcl-gg,HDyKUodeuNw,2,2026-01-23T06:43:39Z,2026-02-14 22:33:26.405853+00:00
3,UUUEOHBht8pnQhQvCfIcl-gg,Zj6uiqMvFOU,3,2026-01-17T20:07:55Z,2026-02-14 22:33:26.405853+00:00
4,UUUEOHBht8pnQhQvCfIcl-gg,RiYjYfMTGvw,4,2026-01-11T23:25:57Z,2026-02-14 22:33:26.405853+00:00


In [11]:
df_playlist_items.dtypes

playlist_id                     str
video_id                        str
position                      int64
added_at                        str
extracted_at    datetime64[us, UTC]
dtype: object

In [12]:
# El added_at viene de la API como string ISO (ya en UTC normalmente con Z), debes convertirlo:
df_playlist_items_uploads["added_at"] = (
    pd.to_datetime(df_playlist_items_uploads["added_at"], utc=True)
)

In [13]:
df_playlist_items.shape

(199, 5)

> ⚠️ Nota -> NO SE GUARDARÁ df_playlist_items_uploads COMO PARQUET PORQUE NO SE UTILIZARÁ DEBIDO A QUE LA PLAYLIST QUE TIENE ES LA MISMA PARA TODAS LOS VIDEOS, EN VEZ DE ESO SE UTILIZARÁ df_playlist_items_manual